In [3]:
from single_nav_analysis import NavAnalysisConfig, SingleNavAnalysis
from pathlib import Path
import numpy as np


nav_analysis_config = NavAnalysisConfig(

    bench_data_path=Path(r"C:\Euclid_Jie\barra\src\nav_analysis\index_data.csv"),
    nav_data_path=Path(
        r"C:/Users/Ouwei/Desktop/nav_data/净值0814/市场中性/天算中性B-SXU256.xlsx"
    ),

    begin_date=np.datetime64("2023-12-29"),
    open_html=False,
    image_save_parh=None,

    benchmark="SHSE.000905",
)
demo = SingleNavAnalysis(nav_analysis_config)
demo.analysis()
demo.export_html()

读取天算中性B-SXU256净值数据成功
原始数据时间区间为：2023-05-18 ~ 2024-08-09
本次统计时间区间为：2023-12-29 ~ 2024-08-09
html路径为：C:\Users\Ouwei\Desktop\nav_data\净值0814\市场中性\20240916_天算中性B-SXU256_nav_analysis.html


In [5]:
demo.backword_analysis_df

,区间收益率,年化收益率,区间波动率,年化波动率,夏普比率,最大回撤,begin_date,end_date
超额_天算中性B-SXU256_1M,1.212%,2.625%,0.298%,2.148%,1.222,-0.965%,2024-07-09,2024-08-09
超额_天算中性B-SXU256_3M,-1.999%,-1.575%,0.465%,3.353%,-0.470,-3.629%,2024-05-09,2024-08-09
超额_天算中性B-SXU256_6M,1.552%,0.678%,0.583%,4.202%,0.161,-6.259%,2024-02-19,2024-08-09
超额_天算中性B-SXU256_YTD,-12.162%,-4.302%,0.814%,5.870%,-0.733,-17.024%,2024-01-02,2024-08-09


In [2]:
from compare_nav_analysis import NavAnalysisConfig, CompareNavAnalysis
from pathlib import Path
import numpy as np

nav_analysis_config = NavAnalysisConfig(
    bench_data_path=Path(r"C:\Euclid_Jie\barra\src\nav_analysis\index_data.csv"),
    nav_data_path=[
        Path(r"C:\Users\Ouwei\Desktop\nav_data\SGB773_麦迪生利锐联中性对冲1号.xlsx"),
        Path(
            r"C:/Users/Ouwei/Desktop/nav_data/净值0814/市场中性/天算中性B-SXU256.xlsx"
        ),
    ],
    begin_date=np.datetime64("2023-12-29"),
    open_html=True,
    image_save_parh=None,
    benchmark="SHSE.000905",
)
demo = CompareNavAnalysis(nav_analysis_config)
demo.anlysis()

SGB773_麦迪生利锐联中性对冲1号，原始数据时间区间为：2023-05-19 - 2024-08-23
天算中性B-SXU256，原始数据时间区间为：2023-05-18 - 2024-08-09
本次统计时间区间为：2023-12-29 ~ 2024-08-09
开始读取SGB773_麦迪生利锐联中性对冲1号净值数据
原始数据时间区间为：2023-05-19 ~ 2024-08-23
本次统计时间区间为：2023-12-29 ~ 2024-08-09
开始读取天算中性B-SXU256净值数据
原始数据时间区间为：2023-05-18 ~ 2024-08-09
本次统计时间区间为：2023-12-29 ~ 2024-08-09


In [3]:
demo.backword_analysis_df

,区间收益率,年化收益率,区间波动率,年化波动率,夏普比率,最大回撤,begin_date,end_date
超额_SGB773_麦迪生利锐联中性对冲1号_1M,2.324%,24.171%,1.750%,12.617%,1.916,-2.298%,2024-07-12,2024-08-09
超额_SGB773_麦迪生利锐联中性对冲1号_3M,18.545%,68.882%,1.632%,11.768%,5.853,-2.298%,2024-05-10,2024-08-09
超额_SGB773_麦迪生利锐联中性对冲1号_6M,15.043%,31.290%,2.122%,15.301%,2.045,-4.833%,2024-02-23,2024-08-09
超额_SGB773_麦迪生利锐联中性对冲1号_YTD,24.676%,41.393%,3.626%,26.148%,1.583,-15.759%,2024-01-05,2024-08-09
超额_天算中性B-SXU256_1M,1.212%,2.625%,0.298%,2.148%,1.222,-0.965%,2024-07-09,2024-08-09
超额_天算中性B-SXU256_3M,-1.999%,-1.575%,0.465%,3.353%,-0.470,-3.629%,2024-05-09,2024-08-09
超额_天算中性B-SXU256_6M,1.552%,0.678%,0.583%,4.202%,0.161,-6.259%,2024-02-19,2024-08-09
超额_天算中性B-SXU256_YTD,-12.162%,-4.302%,0.814%,5.870%,-0.733,-17.024%,2024-01-02,2024-08-09
